In [1]:
from yanytapi import SearchAPI
import pandas as pd
from datetime import datetime
import time

In [2]:
keyword_dict = {
    'S&P': ['S. P.', 'Wall Street', 'Fed', 'Federal Reserve'],
    'AAPL': ['Apple'],
    'MSFT': ['Microsoft'],
    'AMZN': ['Amazon'],
    'TSLA': ['Tesla'],
    'GOOGL': ['Alphabet', 'Google', 'GOOG'],
    'BRK.B': ['Berkshire Hathaway', 'BRK'],
    'UNH': ['UnitedHealth'],
    'JNJ': ['Johnson', 'JnJ'],
    'XOM': ['ExxonMobil', 'Exxon'],
    'JPM': ['JP Morgan', 'JPMorgan'],
    'META': ['Meta', 'Facebook'],
    'NVDA': ['NVIDIA'],
    'PG': ['Procter'], 
    'V': ['Visa'],
    'HD': ['Home Depot'],
    'CVX': ['Chevron Corporation'],
    'MA': ['Mastercard'],
    'PFE': ['Pfizer'],
    'ABBV': ['AbbVie'],
    'BAC': ['Bank of America'],
    'LLY': ['Eli Lilly and Company'],
    'KO': ['Coca-Cola'],
    'PEP': ['PepsiCo', 'Pepsi'],
    'COST': ['Costco'],
    'MRK': ['Merck'], 
    'TMO': ['Thermo Fisher', 'Thermo Fisher Scientific'],
    'AVGO': ['Broadcom'],
    'DIS': ['Walt Disney'],
    'WMT': ['Walmart'],
    'MCD': ['McDonald\'s'],
}

In [3]:
api = SearchAPI("RXkqQPmyy2JObRlC0qFYQj2vNxtX4oxP")

In [4]:
dates = []
keywords = []
headlines = []
abstracts = []
lead_paragraph = []
sections = []
hits = []
word_counts = []

for company, keyword_list in keyword_dict.items():
    print(f'### {company} ###')
    for keyword in keyword_list:
        count = 0
        articles = api.search(keyword, fq={"body": keyword, "source": ["Reuters","AP","The New York Times"]},
                        begin_date="20190101",
                        end_date = "20220901",
                        facet_field=["source", "day_of_week"],
                        facet_filter=True)
        
        for item in articles:
            relevant = False
            if (count % 100 == 0):
                time.sleep(2)

            # Filtering for S&P
            if keyword == 'S. P.':
                sp_keywords = ["s&p", "s.&p.", "standard & poor", "stock", "index", "market"]
                for kw in sp_keywords:
                    if kw in item.headline["main"].lower() or kw in item.abstract.lower():
                        relevant = True
                        break
            else:
                relevant = True

            if relevant:
                count += 1
                dates.append(datetime.strptime(item.pub_date[:10], '%Y-%m-%d'))
                keywords.append(keyword)
                headlines.append(item.headline["main"])
                abstracts.append(item.abstract)
                lead_paragraph.append(item.lead_paragraph)
                sections.append(item.section_name)
                hits.append(item.meta.hits)
                word_counts.append(item.word_count)
        print(f'{keyword}: {count}')
    
    df = pd.DataFrame({'date': dates, 'keyword': keywords, 'headline': headlines, 'abstract': abstracts, 'lead_paragraph':lead_paragraph, 'section': sections, 'hits': hits, 'word_count': word_counts})
    df.set_index('date', inplace=True)
    df = df.sort_index()

    df.to_excel('nyt_2019_2022_raw_data_mcd.xlsx')
    time.sleep(5)

### MCD ###
McDonald's: 1000


In [ ]:
temp = pd.DataFrame(df_no_duplicate.groupby('date').count()['headline'])
temp = temp.rename(columns={'headline': 'article_count'})
temp = temp.reindex(pd.date_range(start="2019-01-01", end="2022-09-01"))
temp = temp.fillna(0)

In [ ]:
test = df_no_duplicate.reset_index()
test[test['date'] == '2020-01-18']

In [ ]:
temp[temp['article_count']==1]

In [ ]:
temp.sort_values('article_count')

In [ ]:
# # combine all files
# df1 = pd.read_excel('nyt_2019_2022_subset_tillNVDA.xlsx')
# df2 = pd.read_excel('nyt_2019_2022_subset_Visa_Costco.xlsx')
# df3 = pd.read_excel('nyt_2019_2022_subset_Thermo_Mcd.xlsx')
# df4 = pd.read_excel('nyt_2019_2022_subset_Procter_Merck.xlsx')

# inv_dict = {}
# for k, v_list in keyword_dict.items():
#     for v in v_list:
#         inv_dict[v] = k

# final_df = pd.concat([df1, df2, df3, df4])
# final_df.set_index('date', inplace=True)
# final_df = final_df.sort_index()
# final_df = final_df.drop_duplicates(subset=['headline'])
# final_df['stock'] = [inv_dict[keyword] for keyword in final_df['keyword']]
# final_df.reset_index()
# final_df.to_excel('nyt_2019_2022.xlsx')

In [2]:
df_2019_2022 = pd.read_excel('nyt_2019_2022_body.xlsx')
df_2016_2018 = pd.read_excel('nyt_2016_2018_body_no_duplicate.xlsx').drop('Unnamed: 0', axis=1)
df = pd.concat([df_2016_2018, df_2019_2022])

In [3]:
filtered_df = df[df['section'].isin(['Blogs', 'Business Day', 'Opinion', 'Technology', 'The Upshot', 'U.S.', 'New York', 'World'])]
print(f'rows filtered: {len(df) - len(filtered_df)}')
print(f'rows left: {len(filtered_df)}')

rows filtered: 9797
rows left: 21851


In [4]:
temp = pd.DataFrame(filtered_df.groupby('date').count()['headline'])
temp = temp.rename(columns={'headline': 'article_count'})
temp = temp.reindex(pd.date_range(start="2016-01-01", end="2022-09-01"))
temp = temp.fillna(0)

In [7]:
zero = len(temp[temp['article_count']==0])
print(f'0 articles: {zero}')

less_five = len(temp[temp['article_count']<5])
print(f'<5 articles: {less_five}')

0 articles: 10
<5 articles: 503


In [6]:
filtered_df.to_excel('nyt_body_filtered_sections.xlsx')

In [4]:
df = pd.read_excel('nyt_2016_2022_new.xlsx')

In [5]:
df

,Unnamed: 0,Unnamed: 0.1,date,keyword,headline,abstract,lead_paragraph,section,hits,word_count
0,0,3,2016-01-01,"['Microsoft', 'Tesla']",Looking Beyond the Internet of Things,"Adam Bosworth, a tech pioneer, sees the future...",SAN FRANCISCO — If you have sent email on Goog...,Technology,1259,1180
1,1,10,2016-01-01,"[""McDonald's""]",No More Statutes of Limitations for Rape,Bill Cosby came close to escaping sexual assau...,"THIS week, Bill Cosby was charged with three c...",Opinion,1514,914
2,2,11,2016-01-01,['Visa'],U.S. Doesn’t Know How Many Foreign Visitors Ov...,After two decades of failed attempts to track ...,WASHINGTON — The question from the congressman...,U.S.,2394,1189
3,3,12,2016-01-02,['Fed'],Making And Using Models,It’s about self-discipline.,"Larry Summers, Brad DeLong, and yours truly ar...",Opinion,3998,576
4,4,15,2016-01-02,['Amazon'],Cutting the Cord and Feeling Good About It,Canceling cable has meant becoming more intent...,"Nearly three years ago, when I first thought a...",Opinion,6215,440
...,...,...,...,...,...,...,...,...,...,...
18734,9564,22952,2022-08-02,"[""McDonald's""]","Starbucks Reports Record Revenue, Driven Mostl...",The largest coffee chain in the world said it ...,Customers flocking to Starbucks and ordering P...,Business Day,1849,343
18735,9565,22956,2022-08-11,"[""McDonald's""]",The Golden Arches Return to Ukraine After a 6-...,McDonald’s said it would begin reopening store...,McDonald’s will begin reopening its restaurant...,Business Day,1849,314
18736,9566,22958,2022-08-15,"[""McDonald's""]",The Two Simple Edicts of Successful Addiction ...,"In the opioid epidemic, citizens step in to he...","HICKORY, N.C. — On a chilly spring evening in ...",Opinion,1849,2171
18737,9567,22961,2022-08-22,"[""McDonald's""]","Under Pressure, McDonald’s Shakes Up Its Board",The fast-food chain has wrestled with some sha...,McDonald’s announced a shake-up of its board o...,Business Day,1849,492


In [6]:
temp = pd.DataFrame(df.groupby('date').count()['headline'])
temp = temp.rename(columns={'headline': 'article_count'})
temp = temp.reindex(pd.date_range(start="2016-01-01", end="2022-09-01"))
temp = temp.fillna(0)

In [7]:
temp.mean()

article_count    7.692529
dtype: float64

In [8]:
zero = len(temp[temp['article_count']==0])
print(f'0 articles: {zero}')

less_five = len(temp[temp['article_count']<5])
print(f'<5 articles: {less_five}')

0 articles: 27
<5 articles: 647
